# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   365.574565 -0.018056  1234  365.556509  365.592622   731.113017   
6   819.900525  0.041602  1234  819.942127  819.858923  1639.884254   
7   731.767864 -0.315567  1234  731.452298  732.083431  1462.904595   
8   546.543586  0.112684  1234  546.656270  546.430902  1093.312539   
9   677.475534 -0.674057  1234  676.801478  678.149591  1353.602955   
10  338.943626  0.751237  1234  339.694863  338.192390   679.389727   
11  727.589913  0.443055  1234  728.032968  727.146859  1456.065936   
12  542.968493 -0.156737  1234  542.811756  543.125230  1085.623512   
13  588.755712 -0.250515  1234  588.505197  589.006227  1177.010394   
14  622.891734 -0.131375  1234  622.760360  623.023109  1245.520719   

              f  
5    731.185243  
6   1639.717846  
7   1464.166862  
8   1092.861805  
9   1356.299182  
10   676.384779  
11  1454.293718  
12  1086.250459  
13  1178.012455  
14  1246.046218

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -365.556509   365.592622
6   -819.942127   819.858923
7   -731.452298   732.083431
8   -546.656270   546.430902
9   -676.801478   678.149591
10  -339.694863   338.192390
11  -728.032968   727.146859
12  -542.811756   543.125230
13  -588.505197   589.006227
14  -622.760360   623.023109

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -365.556509   365.592622
6   -819.942127   819.858923
7   -731.452298   732.083431
8   -546.656270   546.430902
9   -676.801478   678.149591
10  -339.694863   338.192390
11  -728.032968   727.146859
12  -542.811756   543.125230
13  -588.505197   589.006227
14  -622.760360   623.023109

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-365.55650856,  365.59262156],
       [-819.94212721,  819.85892311],
       [-731.45229767,  732.08343117],
       [-546.65626951,  546.43090244],
       [-676.80147754,  678.14959107],
       [-339.69486327,  338.19238967],
       [-728.032968  ,  727.14685895],
       [-542.81175604,  543.12522969],
       [-588.50519682,  589.00622744],
       [-622.76035973,  623.02310903]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5     729.113017   -731.185243
6    1637.884254  -1639.717846
7    1460.904595  -1464.166862
8    1091.312539  -1092.861805
9    1351.602955  -1356.299182
10    677.389727   -676.384779
11   1454.065936  -1454.293718
12   1083.623512  -1086.250459
13   1175.010394  -1178.012455
14   1243.520719  -1246.046218

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf